In [105]:
import pandas as pd

#CHANGE CSV FILES

# Read in CSV Files for companies
apple_df = pd.read_csv('AAPL_Training.csv')
amazon_df = pd.read_csv('AMZN_Training.csv')
google_df = pd.read_csv('GOOG_Training.csv')
microsoft_df = pd.read_csv('MSFT_Training.csv')
tesla_df = pd.read_csv('TSLA_Training.csv')

print(google_df)


            Date       Open       High        Low      Close  Adj Close  \
0     2011-01-03  14.856315  15.083214  14.856315  15.052330  15.052330   
1     2011-01-04  15.083962  15.097909  14.946975  14.996788  14.996788   
2     2011-01-05  14.945730  15.201272  14.945232  15.169890  15.169890   
3     2011-01-06  15.209989  15.403016  15.194298  15.280226  15.280226   
4     2011-01-07  15.340251  15.398533  15.196291  15.353451  15.353451   
...          ...        ...        ...        ...        ...        ...   
2512  2020-12-24  86.750000  87.300003  86.455498  86.942497  86.942497   
2513  2020-12-28  87.581749  89.536400  87.316750  88.804497  88.804497   
2514  2020-12-29  89.389503  89.622002  87.804497  87.935997  87.935997   
2515  2020-12-30  88.100502  88.254753  86.279999  86.975998  86.975998   
2516  2020-12-31  86.771004  87.946503  86.771004  87.594002  87.594002   

         Volume  
0      94962614  
1      73253547  
2     101671667  
3      82620526  
4      84

In [106]:
# Function for calculating FIRST parameter (each day's high - low)

def calculate_high_low_diff(df):
    high_low_values = df["High"] - df["Low"]
    new_df = pd.DataFrame({"Date": df["Date"], "High-Low": high_low_values})
    return new_df



In [107]:
# Variable 1
# New DataFrame for each company's High-Low Variable

apple_high_minus_low = calculate_high_low_diff(apple_df)
amazon_high_minus_low = calculate_high_low_diff(amazon_df)
google_high_minus_low = calculate_high_low_diff(google_df)
microsoft_high_minus_low = calculate_high_low_diff(microsoft_df)
tesla_high_minus_low = calculate_high_low_diff(tesla_df)


In [108]:
# Function for calculating SECOND parameter (each day's close - open)

def calculate_close_open_diff(df):
    new_df = pd.DataFrame()
    new_df['Date'] = df['Date']
    new_df['Close-Open'] = df['Close'] - df['Open']
    return new_df


In [109]:
# Variable 2
# New DataFrame for each company's Close-Open Variable

apple_high_minus_low = calculate_close_open_diff(apple_df)
amazon_high_minus_low = calculate_close_open_diff(amazon_df)
google_high_minus_low = calculate_close_open_diff(google_df)
microsoft_high_minus_low = calculate_close_open_diff(microsoft_df)
tesla_high_minus_low = calculate_close_open_diff(tesla_df)

print(tesla_high_minus_low)


            Date  Close-Open
0     2011-01-03   -0.014666
1     2011-01-04    0.000667
2     2011-01-05    0.023334
3     2011-01-06    0.070000
4     2011-01-07    0.016000
...          ...         ...
2512  2020-12-24    6.259994
2513  2020-12-28   -3.606674
2514  2020-12-29    1.663346
2515  2020-12-30    7.593338
2516  2020-12-31    1.893326

[2517 rows x 2 columns]


In [110]:
# Function for calculating THIRD parameter (7 Day Moving Average)

def moving_average(dataframe, days):
    # Create a new dataframe with the moving average
    ma_df = pd.DataFrame()
    ma_df['Date'] = dataframe['Date']
    ma_df['Moving Avg'] = dataframe['Close'].rolling(window=days).mean()

    # Add a column with the date of each week
    ma_df['Week'] = pd.to_datetime(ma_df['Date']).dt.to_period('W').dt.start_time
    
    # Group by week and get the last date and moving average for each week
    ma_df = ma_df.groupby('Week').tail(1)

    return ma_df


In [111]:
ma_7days = moving_average(apple_df, 5)
ma_7days.set_index('Date', inplace=True)
print(ma_7days.head())


            Moving Avg       Week
Date                             
2011-01-07   11.890786 2011-01-03
2011-01-14   12.304786 2011-01-10
2011-01-21   12.052643 2011-01-17
2011-01-28   12.157214 2011-01-24
2011-02-04   12.275786 2011-01-31


In [112]:
ma_7days = moving_average(apple_df, 15)
ma_7days.set_index('Date', inplace=True)
print(ma_7days)

            Moving Avg       Week
Date                             
2011-01-07         NaN 2011-01-03
2011-01-14         NaN 2011-01-10
2011-01-21         NaN 2011-01-17
2011-01-28   12.142119 2011-01-24
2011-02-04   12.161881 2011-01-31
...                ...        ...
2020-12-04  118.976000 2020-11-30
2020-12-11  120.305333 2020-12-07
2020-12-18  123.895333 2020-12-14
2020-12-24  126.245333 2020-12-21
2020-12-31  129.299334 2020-12-28

[522 rows x 2 columns]


In [99]:
import pandas as pd
from datetime import timedelta

def moving_average(df, weeks):
    # convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # set 'Date' as the index
    df.set_index('Date', inplace=True)
    # resample to weekly frequency, taking the last value of each week
    df_weekly = df.resample('W').last()
    # compute the rolling mean of the 'Close' column for the specified number of weeks
    ma = df_weekly['Close'].rolling(weeks).mean()
    # create a new DataFrame with the moving average
    df_ma = pd.DataFrame({'Date': ma.index, 'Moving Average': ma})
    # forward-fill missing values
    df_ma.fillna(method='ffill', inplace=True)
    # set 'Date' as the index
    df_ma.set_index('Date', inplace=True)
    # join the original DataFrame with the new DataFrame
    df_ma = df.join(df_ma)
    return df_ma


In [101]:
# apple_df
df_ma = moving_average(apple_df, 4) # calculate 4-week moving average
print(df_ma)




                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2011-01-03   11.630000   11.795000   11.601429   11.770357   10.017968   
2011-01-04   11.872857   11.875000   11.719643   11.831786   10.070250   
2011-01-05   11.769643   11.940714   11.767857   11.928571   10.152627   
2011-01-06   11.954286   11.973214   11.889286   11.918929   10.144420   
2011-01-07   11.928214   12.012500   11.853571   12.004286   10.217066   
...                ...         ...         ...         ...         ...   
2020-12-24  131.320007  133.460007  131.100006  131.970001  130.205780   
2020-12-28  133.990005  137.339996  133.509995  136.690002  134.862686   
2020-12-29  138.050003  138.789993  134.339996  134.869995  133.067001   
2020-12-30  135.580002  135.990005  133.399994  133.720001  131.932373   
2020-12-31  134.080002  134.740005  131.720001  132.690002  130.916153   

               Volume   14-Day MA    

In [82]:
moving_average(apple_df, weeks=2)


,Date,Moving Avg,Week
4,2011-01-07,NaN,2011-01-03
9,2011-01-14,12.097786,2011-01-10
13,2011-01-21,12.134571,2011-01-17
18,2011-01-28,12.104929,2011-01-24
23,2011-02-04,12.216500,2011-01-31
...,...,...,...
2498,2020-12-04,118.902000,2020-11-30
2503,2020-12-11,122.560001,2020-12-07
2508,2020-12-18,124.838999,2020-12-14
2512,2020-12-24,127.828001,2020-12-21


In [123]:
import pandas as pd

# Load your dataframe
# apple_df = pd.read_csv('apple.csv')

# Convert the 'Date' column to a datetime object and set it as the index
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df.set_index('Date', inplace=True)

# Resample to weekly frequency, starting on Mondays
weekly_df = apple_df.resample('W-MON').last()

# Calculate the moving average for each week interval, using a window of 2 weeks
weekly_df['2W Moving Avg'] = weekly_df['Close'].rolling(window='2W-MON').mean()


KeyError: 'Date'

In [124]:
import pandas as pd

def two_week_moving_average(data):
    # Convert the 'Date' column to a datetime object and set it as the index
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    
    # Resample to weekly frequency, starting on Mondays
    weekly_data = data.resample('W-MON').last()
    
    # Compute the 2-week moving average
    two_week_ma = weekly_data['Close'].rolling(window=2).mean()
    
    return two_week_ma


In [125]:
two_week_moving_average(apple_df)

KeyError: 'Date'